# Predicción del Precio del Polipropileno con Deep Learning (CNN 1D y LSTM)

Este notebook desarrolla un flujo completo para:

- Obtener datos históricos del precio del **polipropileno (PP)** desde la web (web scraping).
- Explorar y preprocesar la serie de tiempo.
- Construir modelos de **redes neuronales profundas** (LSTM y CNN 1D) para predecir el precio futuro.
- Evaluar el desempeño (MSE, MAPE) y generar **predicciones a futuro** (5–7 días).
- Preparar una tabla de predicciones para fechas específicas (24–28 de noviembre de 2025).


## 1. Introducción

### ¿Qué es el polipropileno?

El polipropileno (PP) es un **polímero termoplástico** muy usado en empaques, textiles, automotriz, construcción, electrodomésticos y muchos otros sectores. Es ligero, resistente a químicos y al calor, y relativamente barato, por eso es de los plásticos más consumidos a nivel mundial.

### ¿Por qué es relevante su precio?

El precio del PP impacta:

- **Costos de producción** de empaques, partes automotrices y bienes de consumo.
- **Márgenes** de fabricantes y transformadores de plástico.
- **Decisiones de compra y planeación** en cadenas de suministro.

Por eso, poder **modelar y predecir** su precio ayuda a:

- Negociar contratos y compras de materia prima.
- Planear inventarios y producción.
- Evaluar riesgos y escenarios de mercado.

### Objetivo del notebook

Usaremos datos históricos diarios del precio del PP para:

1. Construir un dataset limpio con columnas `Date` y `Price`.
2. Analizar el comportamiento del precio en el tiempo.
3. Entrenar modelos **LSTM** y **CNN 1D** para predicción un paso adelante.
4. Evaluar los modelos con **MSE y MAPE** en un conjunto de prueba.
5. Generar predicciones para los próximos días y preparar una tabla con las fechas
   del **24 al 28 de noviembre de 2025**.


## 2. Imports y configuración global

In [67]:

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

from datetime import datetime, timedelta

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense, LSTM, Dropout, Conv1D, Flatten
)
from tensorflow.keras.optimizers import Adam

# Para que las gráficas se muestren bien en Jupyter
pio.renderers.default = "vscode"

# Fijar semillas para algo de reproducibilidad
np.random.seed(63)
tf.random.set_seed(63)

# ---- Parámetros globales del experimento ----

# Fecha mínima opcional para cortar el histórico (string 'YYYY-MM-DD' o None)
START_DATE = '2016-01-01'  # Ejemplo: "2018-01-01"

# Tamaño de ventana (días de histórico que ve el modelo)
WINDOW_SIZE = 60

# Fracciones para splits temporales
TRAIN_FRACTION = 0.7
VAL_FRACTION = 0.15
TEST_FRACTION = 0.15  # se calcula como 1 - TRAIN - VAL si quieres

# Qué modelos entrenar (puedes apagar uno si quieres)
TRAIN_LSTM = True
TRAIN_CNN = True

# Cuántos días hacia adelante predecir en las funciones de forecast
N_FUTURE_DAYS = 5  

# Fechas objetivo para tabla específica (24–28 Nov 2025)
TARGET_DATES_5D = pd.to_datetime(
    ["2025-11-24", "2025-11-25", "2025-11-26", "2025-11-27", "2025-11-28"]
)

In [68]:
# 3. Funciones auxiliares

def mean_absolute_percentage_error(y_true, y_pred):
    """Calcula MAPE en % evitando divisiones raras."""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def create_windowed_dataset(series_1d, window_size):
    """
    Recibe un array 1D (serie escalada) y genera:
    X: [n_samples, window_size]
    y: [n_samples]
    """
    X, y = [], []
    for i in range(len(series_1d) - window_size):
        X.append(series_1d[i : i + window_size])
        y.append(series_1d[i + window_size])
    return np.array(X), np.array(y)


def forecast_future(model, last_window_scaled, scaler, n_future=5):
    """
    Hace forecast recursivo n_future pasos adelante.
    last_window_scaled: array shape (window_size,) ya escalado.
    Devuelve:
      future_scaled: array de tamaño n_future en escala normalizada
      future_inv: array en escala original
    """
    window = last_window_scaled.copy()
    preds_scaled = []

    for _ in range(n_future):
        x_input = window.reshape(1, -1, 1)  # (1, timesteps, features=1)
        pred_scaled = model.predict(x_input, verbose=0)[0, 0]
        preds_scaled.append(pred_scaled)
        # Desplazar ventana y agregar nuevo valor
        window = np.roll(window, -1)
        window[-1] = pred_scaled

    preds_scaled = np.array(preds_scaled)
    preds_inv = scaler.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()
    return preds_scaled, preds_inv


def plot_test_predictions(dates_test, y_test_inv, y_pred_inv, title_prefix="LSTM"):
    """Gráfica real vs predicho solo en test."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_test, y=y_test_inv,
        mode="lines", name="Real (test)"
    ))
    fig.add_trace(go.Scatter(
        x=dates_test, y=y_pred_inv,
        mode="lines", name="Predicho (test)"
    ))
    fig.update_layout(
        title=f"{title_prefix} - Precio real vs predicho (set de prueba)",
        xaxis_title="Fecha",
        yaxis_title="Precio"
    )
    fig.show()


def plot_full_predictions(dates_all, y_all_inv, y_pred_all_inv, title_prefix="LSTM"):
    """Gráfica real vs predicho para todo el histórico (train+val+test)."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_all, y=y_all_inv,
        mode="lines", name="Real (precio spot)"
    ))
    fig.add_trace(go.Scatter(
        x=dates_all, y=y_pred_all_inv,
        mode="lines", name="Predicho"
    ))
    fig.update_layout(
        title=f"{title_prefix} - Real vs predicción en todo el histórico",
        xaxis_title="Fecha",
        yaxis_title="Precio"
    )
    fig.show()


def plot_full_with_future(dates_all, y_all_inv, future_dates, future_inv, title_prefix="LSTM"):
    """Histórico completo + predicciones futuras."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_all, y=y_all_inv,
        mode="lines", name="Real (histórico)"
    ))
    fig.add_trace(go.Scatter(
        x=future_dates, y=future_inv,
        mode="lines+markers", name="Predicción futura"
    ))
    fig.update_layout(
        title=f"{title_prefix} - Histórico completo + predicción de los próximos días",
        xaxis_title="Fecha",
        yaxis_title="Precio"
    )
    fig.show()

### Descarga Data

In [69]:
from alphacast import Alphacast
#%pip install alphacast
alphacast = Alphacast("ak_eX9r2t13iRJ6uDIbKzD1")

# df_raw = alphacast.datasets.dataset(40971).download_data("pandas")
# df_raw.to_parquet("polipropileno_data.parquet")
df_raw = pd.read_parquet("polipropileno_data.parquet")

df_raw = df_raw[df_raw['Industrial'] == 'Polypropylene']
df_raw["Date"] = pd.to_datetime(df_raw["Date"])
df_raw = df_raw.sort_values("Date")
df_raw = df_raw[["Date", "Price"]]
df_raw = df_raw[df_raw["Date"] >= START_DATE]
df_raw = df_raw.reset_index(drop=True)
df = df_raw
df


,Date,Price
0,2016-01-04,6387.0
1,2016-01-05,6351.0
2,2016-01-06,6343.0
3,2016-01-07,6143.0
4,2016-01-08,6007.0
...,...,...
2294,2025-11-11,6422.0
2295,2025-11-12,6411.0
2296,2025-11-13,6407.0
2297,2025-11-14,6446.0


In [ ]:
# Gráfica básica del precio vs tiempo (histórico completo descargado)

fig = px.line(
    df,
    x="Date",
    y="Price",
    title="Precio del Polipropileno (Futuros) - Histórico (sin recorte)"
)
fig.update_xaxes(title="Fecha")
fig.update_yaxes(title="Precio (unidades del sitio fuente)")
fig.show()


## 3. Revisión del dataset

En esta sección:

- Revisamos la estructura del DataFrame (`info`, `describe`).
- Vemos el rango de fechas disponible.
- Agregamos un **corte opcional** a partir de una fecha (`START_DATE`) por si queremos trabajar
  solo con una ventana más reciente.
- Graficamos:
  - Precio vs tiempo (recortado).
  - Precio + media móvil de 30 días.
  - Histograma de precios.


In [ ]:
# Info general del dataset
print("Shape (filas, columnas):", df.shape)
print("\nInfo:")
print(df.info())

print("\nDescribe:")
display(df["Price"].describe())

print("\nRango de fechas disponible:")
print("Mínima:", df["Date"].min())
print("Máxima:", df["Date"].max())

# ---- Corte opcional por fecha ----
if START_DATE is not None:
    start_dt = pd.to_datetime(START_DATE)
    df = df[df["Date"] >= start_dt].copy()
    print(f"\nAplicando corte desde {start_dt.date()}:")
else:
    df = df.copy()
    print("\nSin corte de fecha, usando todo el histórico.")

print("Shape después del corte:", df.shape)
print("Nuevo rango de fechas:")
print("Mínima:", df["Date"].min())
print("Máxima:", df["Date"].max())


Shape (filas, columnas): (2299, 2)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2299 entries, 0 to 2298
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2299 non-null   datetime64[ns]
 1   Price   2299 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 36.0 KB
None

Describe:


count     2299.000000
mean      8144.671596
std        911.487530
min       5874.000000
25%       7489.000000
50%       8119.000000
75%       8744.000000
max      11168.000000
Name: Price, dtype: float64


Rango de fechas disponible:
Mínima: 2016-01-04 00:00:00
Máxima: 2025-11-17 00:00:00

Aplicando corte desde 2016-01-01:
Shape después del corte: (2299, 2)
Nuevo rango de fechas:
Mínima: 2016-01-04 00:00:00
Máxima: 2025-11-17 00:00:00


In [72]:
# Agregar media móvil de 30 días
df["MA_30"] = df["Price"].rolling(window=30, min_periods=1).mean()

# 1) Precio vs tiempo (recortado)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df["Date"], y=df["Price"],
    mode="lines", name="Precio diario"
))
fig.update_layout(
    title="Precio diario del PP (posible histórico recortado)",
    xaxis_title="Fecha",
    yaxis_title="Precio"
)
fig.show()

# 2) Precio + media móvil 30 días
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df["Date"], y=df["Price"],
    mode="lines", name="Precio diario"
))
fig.add_trace(go.Scatter(
    x=df["Date"], y=df["MA_30"],
    mode="lines", name="Media móvil 30 días"
))
fig.update_layout(
    title="Precio del PP con media móvil de 30 días",
    xaxis_title="Fecha",
    yaxis_title="Precio"
)
fig.show()

# 3) Histograma de precios
fig = px.histogram(
    df,
    x="Price",
    nbins=30,
    title="Distribución de precios del PP"
)
fig.update_xaxes(title="Precio")
fig.update_yaxes(title="Frecuencia")
fig.show()


## 4. Preprocesamiento

Pasos:

1. Normalizar la columna de precio (`MinMaxScaler`) para estabilizar el entrenamiento.
2. Crear un dataset supervisado usando ventanas deslizantes de tamaño `WINDOW_SIZE`:
   - Entrada: precios de los últimos `WINDOW_SIZE` días.
   - Salida: precio del día siguiente.
3. Dividir en **train / validation / test** respetando el orden temporal.
4. Darle forma a los tensores para que funcionen con LSTM y CNN 1D:
   - Shape: `(n_samples, timesteps, features)` con `features = 1`.


In [73]:
# 4.1 Normalización de la serie

prices = df["Price"].values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
prices_scaled = scaler.fit_transform(prices).flatten()

# 4.2 Crear ventanas (X, y)

X_all, y_all = create_windowed_dataset(prices_scaled, WINDOW_SIZE)

# Fechas para cada y (la predicción se asocia al último día de la ventana)
dates_all = df["Date"].values[WINDOW_SIZE:]

print("Total muestras (ventanas):", X_all.shape[0])

# 4.3 División temporal train / val / test

n_samples = X_all.shape[0]
train_end = int(n_samples * TRAIN_FRACTION)
val_end = int(n_samples * (TRAIN_FRACTION + VAL_FRACTION))

X_train = X_all[:train_end]
y_train = y_all[:train_end]
dates_train = dates_all[:train_end]

X_val = X_all[train_end:val_end]
y_val = y_all[train_end:val_end]
dates_val = dates_all[train_end:val_end]

X_test = X_all[val_end:]
y_test = y_all[val_end:]
dates_test = dates_all[val_end:]

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# 4.4 Dar forma para modelos secuenciales (LSTM / CNN1D)
# Añadimos dimensión de features (=1)

X_train_seq = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_seq = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test_seq = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (WINDOW_SIZE, 1)
print("Input shape para el modelo:", input_shape)


Total muestras (ventanas): 2239
Train: (1567, 60) Val: (336, 60) Test: (336, 60)
Input shape para el modelo: (60, 1)


## 5. Arquitecturas del modelo (LSTM y CNN 1D)

A continuación se definen dos modelos base:

- **Modelo LSTM**:
  - Pensado para capturar dependencias temporales a largo plazo.
  - Arquitectura sencilla: 1–2 capas LSTM + Dropout + capa densa final.

- **Modelo CNN 1D**:
  - Usa convoluciones 1D sobre la serie para aprender patrones locales.
  - Arquitectura sencilla: varias capas Conv1D + Dropout + Flatten + Dense.

También se muestra una tabla con los hiperparámetros probados para cada modelo
(estos valores los puedes ajustar libremente para tus experimentos).


### 5.3 Modelo LSTM (ejemplo sencillo)

En esta sección entrenamos un modelo LSTM base:

- Ventana de `WINDOW_SIZE` días de entrada.
- Dos capas LSTM de 64 unidades con Dropout.
- Función de pérdida: MSE.
- Métricas reportadas: MSE y MAPE en el conjunto de prueba.

Luego generamos:

1. Gráfica de **real vs predicho** en test.
2. Gráfica de **real vs predicho en todo el histórico** (train+val+test).
3. Gráfica de **histórico completo + predicción de los próximos `N_FUTURE_DAYS` días**.


In [74]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Forma de entrada: (window_size, n_features)
input_shape = X_train_seq.shape[1:]  # (WINDOW_SIZE, 1)

# 1) Definición capa a capa
lstm_model = Sequential()

# LSTM 1: capta patrones a ~ventana completa
lstm_model.add(
    LSTM(
        units=32,               # más pequeño que 64, acorde al tamaño de datos
        return_sequences=True,  # deja secuencia para la siguiente LSTM
        input_shape=input_shape
    )
)
lstm_model.add(Dropout(0.15))   # regularización moderada

# LSTM 2: resume la secuencia en un estado final
lstm_model.add(
    LSTM(
        units=16,
        return_sequences=False
    )
)
lstm_model.add(Dropout(0.15))

# Capa densa intermedia: mezcla no lineal
lstm_model.add(Dense(16, activation="relu"))

# Salida: precio del día siguiente
lstm_model.add(Dense(1))

# 2) Compilación
lstm_model.compile(
    optimizer=Adam(learning_rate=5e-4),
    loss="mse"
)

lstm_model.summary()

# 3) Entrenamiento
history_lstm = lstm_model.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=80,
    batch_size=32,
    verbose=1
)

# =========================
#     EVALUACIÓN TEST
# =========================

# Predicciones (escala normalizada)
y_test_pred_scaled_lstm = lstm_model.predict(X_test_seq).flatten()

# Invertir escala
y_test_inv_lstm = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
y_test_pred_inv_lstm = scaler.inverse_transform(
    y_test_pred_scaled_lstm.reshape(-1, 1)
).flatten()

# Métricas
mse_lstm = mean_squared_error(y_test_inv_lstm, y_test_pred_inv_lstm)
mape_lstm = mean_absolute_percentage_error(y_test_inv_lstm, y_test_pred_inv_lstm)

print(f"\n[ LSTM ] MSE (test):  {mse_lstm:.4f}")
print(f"[ LSTM ] MAPE (test): {mape_lstm:.2f}%")

# =========================
#       GRÁFICAS
# =========================

# 1) Real vs predicho solo en test
plot_test_predictions(
    dates_test,
    y_test_inv_lstm,
    y_test_pred_inv_lstm,
    title_prefix="LSTM"
)

# 2) Real vs predicho en todo el histórico (train + val + test)
y_all_pred_scaled_lstm = lstm_model.predict(
    np.concatenate([X_train_seq, X_val_seq, X_test_seq], axis=0)
).flatten()

y_all_inv_lstm = scaler.inverse_transform(
    np.concatenate([y_train, y_val, y_test]).reshape(-1, 1)
).flatten()
y_all_pred_inv_lstm = scaler.inverse_transform(
    y_all_pred_scaled_lstm.reshape(-1, 1)
).flatten()

dates_all_full_lstm = np.concatenate([dates_train, dates_val, dates_test])

plot_full_predictions(
    dates_all_full_lstm,
    y_all_inv_lstm,
    y_all_pred_inv_lstm,
    title_prefix="LSTM"
)

# 3) Histórico completo + predicción de los próximos N_FUTURE_DAYS días
last_window_scaled_lstm = prices_scaled[-WINDOW_SIZE:]
future_scaled_lstm, future_inv_lstm = forecast_future(
    lstm_model,
    last_window_scaled_lstm,
    scaler,
    n_future=N_FUTURE_DAYS
)

last_date = df["Date"].max()
future_dates_lstm = pd.date_range(
    start=last_date + pd.Timedelta(days=1),
    periods=N_FUTURE_DAYS,
    freq="D"
)

plot_full_with_future(
    dates_all_full_lstm,
    y_all_inv_lstm,
    future_dates_lstm,
    future_inv_lstm,
    title_prefix="LSTM"
)

# Tabla de predicciones futuras (LSTM)
future_table_lstm = pd.DataFrame({
    "Fecha futura": future_dates_lstm,
    "Predicción del precio (LSTM)": future_inv_lstm
})
display(future_table_lstm)


c:\Users\esteb\miniconda3\envs\deep_learning\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 60, 32)         │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 60, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 16)             │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,777 (30.38 KB)

 Trainable params: 7,777 (30.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - loss: 0.1323 - val_loss: 0.0081
Epoch 2/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0144 - val_loss: 0.0034
Epoch 3/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.0104 - val_loss: 0.0014
Epoch 4/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.0078 - val_loss: 9.6665e-04
Epoch 5/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0074 - val_loss: 0.0010
Epoch 6/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0070 - val_loss: 9.9391e-04
Epoch 7/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0067 - val_loss: 0.0010
Epoch 8/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0064 - val_loss: 9.2884e-04
Epoch 9/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0063 - val_loss: 0.0012
Epoch 10/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0056 - val_loss: 8.7593e-04
Epoch 11/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0056 - val_loss: 0.0011
Epoch 12/80
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/

70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step


,Fecha futura,Predicción del precio (LSTM)
0,2025-11-18,6552.057617
1,2025-11-19,6573.307617
2,2025-11-20,6598.613770
3,2025-11-21,6623.845215
4,2025-11-22,6646.728027


### 5.4 Modelo CNN 1D (ejemplo sencillo)

Ahora repetimos el flujo con una **red convolucional 1D**:

- Varias capas `Conv1D` para capturar patrones locales en la serie.
- Capa `Flatten` y `Dense` para producir la predicción.
- Igual que en LSTM, medimos MSE y MAPE en test y generamos 3 gráficas:
  1. Real vs predicho (test).
  2. Real vs predicho (todo el histórico).
  3. Histórico + predicción futura.


In [75]:
# =========================
#    MODELO CNN 1D BÁSICO
# =========================

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

input_shape = X_train_seq.shape[1:]  # (WINDOW_SIZE, 1)

# 1) Definición del modelo CNN 1D capa a capa
cnn_model = Sequential()

# Conv1D 1
cnn_model.add(
    Conv1D(
        filters=64,
        kernel_size=3,
        activation="relu",
        input_shape=input_shape
    )
)
cnn_model.add(Dropout(0.2))

# Conv1D 2
cnn_model.add(
    Conv1D(
        filters=64,
        kernel_size=3,
        activation="relu"
    )
)
cnn_model.add(Dropout(0.2))

# Aplanar + densas finales
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation="relu"))
cnn_model.add(Dense(1))

# 2) Compilación
cnn_model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss="mse"
)

cnn_model.summary()

# 3) Entrenamiento
history_cnn = cnn_model.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=30,
    batch_size=32,
    verbose=1
)

# =========================
#     EVALUACIÓN TEST
# =========================

y_test_pred_scaled_cnn = cnn_model.predict(X_test_seq).flatten()

y_test_inv_cnn = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
y_test_pred_inv_cnn = scaler.inverse_transform(
    y_test_pred_scaled_cnn.reshape(-1, 1)
).flatten()

mse_cnn = mean_squared_error(y_test_inv_cnn, y_test_pred_inv_cnn)
mape_cnn = mean_absolute_percentage_error(y_test_inv_cnn, y_test_pred_inv_cnn)

print(f"\n[ CNN1D ] MSE (test):  {mse_cnn:.4f}")
print(f"[ CNN1D ] MAPE (test): {mape_cnn:.2f}%")

# =========================
#       GRÁFICAS
# =========================

# 1) Real vs predicho (test)
plot_test_predictions(
    dates_test,
    y_test_inv_cnn,
    y_test_pred_inv_cnn,
    title_prefix="CNN 1D"
)

# 2) Real vs predicho (train + val + test)
y_all_pred_scaled_cnn = cnn_model.predict(
    np.concatenate([X_train_seq, X_val_seq, X_test_seq], axis=0)
).flatten()

y_all_inv_cnn = scaler.inverse_transform(
    np.concatenate([y_train, y_val, y_test]).reshape(-1, 1)
).flatten()
y_all_pred_inv_cnn = scaler.inverse_transform(
    y_all_pred_scaled_cnn.reshape(-1, 1)
).flatten()

dates_all_full_cnn = np.concatenate([dates_train, dates_val, dates_test])

plot_full_predictions(
    dates_all_full_cnn,
    y_all_inv_cnn,
    y_all_pred_inv_cnn,
    title_prefix="CNN 1D"
)

# 3) Histórico completo + predicción futura
last_window_scaled_cnn = prices_scaled[-WINDOW_SIZE:]
future_scaled_cnn, future_inv_cnn = forecast_future(
    cnn_model,
    last_window_scaled_cnn,
    scaler,
    n_future=N_FUTURE_DAYS
)

last_date = df["Date"].max()
future_dates_cnn = pd.date_range(
    start=last_date + pd.Timedelta(days=1),
    periods=N_FUTURE_DAYS,
    freq="D"
)

plot_full_with_future(
    dates_all_full_cnn,
    y_all_inv_cnn,
    future_dates_cnn,
    future_inv_cnn,
    title_prefix="CNN 1D"
)

# Tabla de predicciones futuras (CNN 1D)
future_table_cnn = pd.DataFrame({
    "Fecha futura": future_dates_cnn,
    "Predicción del precio (CNN 1D)": future_inv_cnn
})
display(future_table_cnn)


c:\Users\esteb\miniconda3\envs\deep_learning\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 58, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 58, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 56, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 56, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 3584)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │       229,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 242,113 (945.75 KB)

 Trainable params: 242,113 (945.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 0.0365 - val_loss: 0.0013
Epoch 2/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0065 - val_loss: 0.0010
Epoch 3/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0053 - val_loss: 4.4979e-04
Epoch 4/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0041 - val_loss: 0.0014
Epoch 5/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0044 - val_loss: 0.0014
Epoch 6/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0037 - val_loss: 2.5291e-04
Epoch 7/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0028 - val_loss: 4.7557e-04
Epoch 8/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0027 - val_loss: 2.4909e-04
Epoch 9/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0025 - val_loss: 5.6674e-04
Epoch 10/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0026 - val_loss: 5.9441e-04
Epoch 11/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0022 - val_loss: 3.7793e-04
Epoch 12/30
49/49 ━━━━━━━━━━━━━━━━━

70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


,Fecha futura,Predicción del precio (CNN 1D)
0,2025-11-18,6585.017578
1,2025-11-19,6603.901367
2,2025-11-20,6592.523438
3,2025-11-21,6585.914551
4,2025-11-22,6586.892578


## 6. Tabla de predicción para fechas específicas (24–28 Nov 2025)

La consigna del proyecto pide una tabla como:

Fecha futura | Predicción del precio ($/ton) | Comentarios

Aquí preparamos la estructura. Tú puedes reemplazar la columna de predicción con los
valores de tu **mejor modelo** (LSTM o CNN 1D), usando el forecast que generes.


In [76]:
# Tabla vacía con la estructura sugerida para el reporte final

tabla_5d = pd.DataFrame({
    "Fecha futura": TARGET_DATES_5D,
    "Predicción del precio ($/ton)": [np.nan] * len(TARGET_DATES_5D),
    "Comentarios": ["—"] * len(TARGET_DATES_5D)
})

tabla_5d


,Fecha futura,Predicción del precio ($/ton),Comentarios
0,2025-11-24,NaN,—
1,2025-11-25,NaN,—
2,2025-11-26,NaN,—
3,2025-11-27,NaN,—
4,2025-11-28,NaN,—
